In [71]:
import csv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import re
import math

In [72]:
df= pd.read_csv('data/2019Transcriptions.csv')

In [73]:
df.head()

,Transcriber,Box #,Block #,Parcel #,Ward #,Street number,Street address,Street number 2 (Optional),Street address 2 (Optional),Land use,...,Award Amount Money,Award Notes,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner\nor\nTenant,Tenant Notes,End of Case Finalized Date,Unnamed: 45
0,NaN,56,1,2,4,198-198½\n,Hillard Ave.,-,-,Commercial,...,-,-,-,-,-,-,-,-,6/6/1969,NaN
1,NaN,56,1,3,4,202-208,Hillard Ave.\n\n,-,-,Commercial,...,21600,There was an interest amount of $536.39. The d...,-,-,-,-,-,-,6/17/1971,NaN
2,NaN,56,1,4,4,90,Asheland Ave.,196,Hillard Ave.,Commercial,...,46000,"There was an interest amount of $1,778.68. The...",-,-,-,-,-,-,7/13/1972,NaN
3,NaN,56,1,5,4,94,Asheland Ave.,-,-,Residential,...,-,-,-,-,-,-,-,-,5/29/1974,NaN
4,NaN,56,1,6,4,96-108,Asheland Ave.,-,-,-,...,-,The award date and amount are unclear because ...,-,-,-,-,-,-,10/9/1974,NaN


In [74]:
copy=df
copy.head()
# "copy" is now the working copy

,Transcriber,Box #,Block #,Parcel #,Ward #,Street number,Street address,Street number 2 (Optional),Street address 2 (Optional),Land use,...,Award Amount Money,Award Notes,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner\nor\nTenant,Tenant Notes,End of Case Finalized Date,Unnamed: 45
0,NaN,56,1,2,4,198-198½\n,Hillard Ave.,-,-,Commercial,...,-,-,-,-,-,-,-,-,6/6/1969,NaN
1,NaN,56,1,3,4,202-208,Hillard Ave.\n\n,-,-,Commercial,...,21600,There was an interest amount of $536.39. The d...,-,-,-,-,-,-,6/17/1971,NaN
2,NaN,56,1,4,4,90,Asheland Ave.,196,Hillard Ave.,Commercial,...,46000,"There was an interest amount of $1,778.68. The...",-,-,-,-,-,-,7/13/1972,NaN
3,NaN,56,1,5,4,94,Asheland Ave.,-,-,Residential,...,-,-,-,-,-,-,-,-,5/29/1974,NaN
4,NaN,56,1,6,4,96-108,Asheland Ave.,-,-,-,...,-,The award date and amount are unclear because ...,-,-,-,-,-,-,10/9/1974,NaN


In [75]:
copy.drop(['Transcriber'], axis=1, inplace=True)

In [76]:
copy.head()

,Box #,Block #,Parcel #,Ward #,Street number,Street address,Street number 2 (Optional),Street address 2 (Optional),Land use,Address Notes,...,Award Amount Money,Award Notes,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner\nor\nTenant,Tenant Notes,End of Case Finalized Date,Unnamed: 45
0,56,1,2,4,198-198½\n,Hillard Ave.,-,-,Commercial,-,...,-,-,-,-,-,-,-,-,6/6/1969,NaN
1,56,1,3,4,202-208,Hillard Ave.\n\n,-,-,Commercial,-,...,21600,There was an interest amount of $536.39. The d...,-,-,-,-,-,-,6/17/1971,NaN
2,56,1,4,4,90,Asheland Ave.,196,Hillard Ave.,Commercial,-,...,46000,"There was an interest amount of $1,778.68. The...",-,-,-,-,-,-,7/13/1972,NaN
3,56,1,5,4,94,Asheland Ave.,-,-,Residential,-,...,-,-,-,-,-,-,-,-,5/29/1974,NaN
4,56,1,6,4,96-108,Asheland Ave.,-,-,-,-,...,-,The award date and amount are unclear because ...,-,-,-,-,-,-,10/9/1974,NaN


#  Parcels Table


In [91]:
parcels=df[['Block #', 'Parcel #','Ward #','Land use']]
# pulling in attributes for Parcels Entity 

In [92]:
parcels=parcels[parcels['Block #'].str.contains("  ")==False]

In [93]:
parcels['Block #'].dtypes

dtype('O')

In [94]:
parcels.insert(0,'Parcel_ID', range(1,1+len(parcels)))

In [95]:
parcels.head()

,Parcel_ID,Block #,Parcel #,Ward #,Land use
0,1,1,2,4,Commercial
1,2,1,3,4,Commercial
2,3,1,4,4,Commercial
3,4,1,5,4,Residential
4,5,1,6,4,-


In [96]:
parcels.columns = ["parcel_id", "block_num", "parcel_num", "ward_num", "land_use"]

In [97]:
parcels.to_csv('output/parcels.csv', index=False)

In [98]:
parcels. shape

(937, 5)

In [99]:
parcel_id=parcels[['parcel_id']]


In [100]:
# pulling attributes for the Address entity
addresses= copy[['Street number', 'Street address']]

In [101]:
# connecting to "parcel_id" as an attribute in the Address entity since it will be the forigen key to Parcels Entity
addresses=addresses.join(parcels[['parcel_id']])

In [102]:
addresses.head()

,Street number,Street address,parcel_id
0,198-198½\n,Hillard Ave.,1.0
1,202-208,Hillard Ave.\n\n,2.0
2,90,Asheland Ave.,3.0
3,94,Asheland Ave.,4.0
4,96-108,Asheland Ave.,5.0


In [103]:
addresses.shape

(939, 3)

In [104]:
addresses.head()

,Street number,Street address,parcel_id
0,198-198½\n,Hillard Ave.,1.0
1,202-208,Hillard Ave.\n\n,2.0
2,90,Asheland Ave.,3.0
3,94,Asheland Ave.,4.0
4,96-108,Asheland Ave.,5.0


In [105]:
# function to take in an object (street num) and split the rows 
def splitRow(table):
#     columns are the values from the colunms of the object passed
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
#         if there is a break within the street num, split the strings then add another row withing that particular cell
        if '\n' in str(row['Street number']):
            data = row['Street number'].split('\n')
            for d in data:
                newrow = row
                newrow['Street number'] = d
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            size+=1
    return newframe

In [106]:
#spliting street num and addresses, now "test" is our modified and split address object
test= splitRow(addresses)
test
##s = test[test['Street address'] == 'Hillard Ave.']
##x = addresses[addresses['Street address'] == 'Hillard Ave.']
##print(x)


C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Street number,Street address,parcel_id
0,198-198½,Hillard Ave.,1.0
1,,Hillard Ave.,1.0
2,202-208,Hillard Ave.\n\n,2.0
3,90,Asheland Ave.,3.0
4,94,Asheland Ave.,4.0
...,...,...,...
943,153,Blanton St.,935.0
944,NS of,Smith St.,936.0
945,171,Blanton St.,937.0
946,NaN,NaN,NaN


In [107]:
test.insert(0,'ID',range(1,1+len(test)))

In [108]:
test.head()

,ID,Street number,Street address,parcel_id
0,1,198-198½,Hillard Ave.,1.0
1,2,,Hillard Ave.,1.0
2,3,202-208,Hillard Ave.\n\n,2.0
3,4,90,Asheland Ave.,3.0
4,5,94,Asheland Ave.,4.0


In [109]:
# setting the datatype for parcel_id as int
test.parcel_id = test["parcel_id"]

In [110]:
test.columns = ["id", "st_num", "st_name", "parcel_id"]

In [61]:
# saving the Entity address
test.to_csv('output/addresses.csv', index=False)

In [111]:
test.head()

,id,st_num,st_name,parcel_id
0,1,198-198½,Hillard Ave.,1.0
1,2,,Hillard Ave.,1.0
2,3,202-208,Hillard Ave.\n\n,2.0
3,4,90,Asheland Ave.,3.0
4,5,94,Asheland Ave.,4.0


# People Table

In [112]:
# retrieving attributes for the People Entity
people=copy[['Previous Transfer of Deed People Involved','Decision People Involved','Final Transfer of Deed People Involved','Appraiser Name','Offer Made People Involved']]

In [113]:
people.head()

,Previous Transfer of Deed People Involved,Decision People Involved,Final Transfer of Deed People Involved,Appraiser Name,Offer Made People Involved
0,Bessie S. Kalogerakis\n\nAmelia K. Kalogerakis...,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...,Eva K. Cledaras\nGeorge M. Cledaras\nSteve Kal...,Clyde G. Idol Jr.\nAlan Butterworth\nH.E. Johnson,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...
1,W.S. Harrison\nBeulah L. Harrison\n\nCharles R...,Charles R. Stinnette Co.\n\nHACA,Charles R. Stinnette Co.\n\nHACA,Charles W. Bradshaw\nAlan Butterworth\nH.E. Jo...,Charles R. Stinnette Co.\n\nHACA
2,Charles G. Lee Sr.\n\nCharles G. Lee Jr.\nH. K...,William I. Lee\nZyphin J. Lee\nH. Kenneth Lee\...,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Charles W. Bradshaw\nAlan Butterworth\nH.E. Jo...,Willian I. Lee\nZyphin J. Lee\nH. Kenneth Lee\...
3,John E. Smathers\nEtta C. Smathers\n\nThelma S...,Thelma Smathers Johnson\nElizabeth Pauline Sma...,Thelma Smathers Johnson\n\nHACA,Clyde G. Idol Jr.\nAlan Butterworth\nH.E. Johnson,Thelma Smathers Johnson\nElizabeth Pauline Sma...
4,C.A. Yarbrough\nSarah M. Yarbrough\n\nHarry's ...,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,Edward Sale\nRoy Moore\nH.E. Johnson,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n


In [114]:
# converting the rows into colunmns 
people = people.stack().reset_index()

In [115]:
people.head()

,level_0,level_1,0
0,0,Previous Transfer of Deed People Involved,Bessie S. Kalogerakis\n\nAmelia K. Kalogerakis...
1,0,Decision People Involved,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...
2,0,Final Transfer of Deed People Involved,Eva K. Cledaras\nGeorge M. Cledaras\nSteve Kal...
3,0,Appraiser Name,Clyde G. Idol Jr.\nAlan Butterworth\nH.E. Johnson
4,0,Offer Made People Involved,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...


In [116]:
people.columns = ["a","b","names"]

In [117]:
# dropping level_0 and level_1 (title) 
people = people.drop(['a', 'b'], axis=1)

In [118]:
# function to take in an object and split the rows 
def splitRows(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
        print(row)
        if '\n' in str(row['names']):
            names = row['names'].split('\n')
            print(names)
            for n in names:
                print (n)
                newrow = row
                newrow['names'] = n
                newframe.loc[size]=newrow
                size+=1
                print(newrow)
        else:
            newframe.loc[size]=row
            size+=1
        break
#     print("final:")
#     print (newframe)
    return newframe

In [119]:
def name_cleaner(name): 
    k = re.sub(r'[0-9][.]', '', name)
    k = re.sub(r'[\(\)\-\&]', '', k)
    k = re.sub(r'[0-9]', '', k)
    k = re.sub(r'[.,]', ' ', k)
    k = re.sub('  ', ' ', k)
    k = k.strip()
    return k

In [120]:
def date_cleaner(date):
    k = re.sub(r'[0-9][.]', '', date)
    k = k.strip()
    return k

In [121]:

def name_organizer(table):
    clean_names=[]
#     looping through the same row 
    for index, row in table.iterrows():
        names = str(row['names'])
        
        if '\n' in names:
            arr = names.split('\n')
            arr = [i for i in arr if i is not ""]
            for n in arr:
                k = name_cleaner(n)
                if k == '': 
                    continue
                clean_names.append(k)
        else:
            k = name_cleaner(names)
            if k == '': 
                continue
            clean_names.append(k)
    return clean_names

In [122]:
# passing people through the function name_organizer to get multiple rows of peoples names 
people=name_organizer(people)

In [123]:
# newframe= declaring a new table with a column "name" 
newframe=pd.DataFrame(columns=['name'])


In [124]:
# assigning each name to the table newframe 
newframe['name'] = pd.Series(people)
newframe.head()

,name
0,Bessie S Kalogerakis
1,Amelia K Kalogerakis
2,Eva K Cleraderas
3,Steve Kalogerakis
4,Amelia K Kalogerakis


In [125]:
newframe = newframe.sort_values(by='name', ascending=True)
newframe.head()

,name
6231,#NAME?
4475,*Not Legable
5398,A Blomberg
13063,A C Brandl
13064,A C Brandl Jr


In [126]:
newframe['id'] = range(len(newframe['name']))
newframe.head()

,name,id
6231,#NAME?,0
4475,*Not Legable,1
5398,A Blomberg,2
13063,A C Brandl,3
13064,A C Brandl Jr,4


In [127]:
newframe.drop(index=0)

,name,id
6231,#NAME?,0
4475,*Not Legable,1
5398,A Blomberg,2
13063,A C Brandl,3
13064,A C Brandl Jr,4
...,...,...
9546,lamar Gudger,14036
9551,lamar Gudger,14037
8965,tecora Cunnigham,14038
8956,tecora Cunnigham,14039


In [128]:
newframe.index = range(0,len(newframe))

In [129]:
newframe.columns = ["name", "person_id"]

In [130]:
newframe["person_id"] = newframe.index + 1

In [131]:
newframe.head()

,name,person_id
0,#NAME?,1
1,*Not Legable,2
2,A Blomberg,3
3,A C Brandl,4
4,A C Brandl Jr,5


In [84]:
newframe.to_csv('output/people_count.csv')

## Event Association Table

In [132]:
event_assoc=copy[['Decision People Involved','Decision Role']]

In [133]:
event_assoc.head()

,Decision People Involved,Decision Role
0,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...,Owner\n\nPurchaser
1,Charles R. Stinnette Co.\n\nHACA,Owner\n\nPurchaser
2,William I. Lee\nZyphin J. Lee\nH. Kenneth Lee\...,Owner\n\nPurchaser
3,Thelma Smathers Johnson\nElizabeth Pauline Sma...,Owner\n\nPurchaser
4,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,Owner\n\nPurchaser


In [134]:
def splitRowEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
        if '\n' in str(row['Decision People Involved']):
            data = row['Decision People Involved'].split('\n')
            roles= str(row['Decision Role']).split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            if('Heir' in roles):
                norm='Heir'
                norm_prev='Heir'
            for d in range(len(data)):
                newrow = row
                newrow['Decision People Involved'] = data[d]
                if data[d]!='':
                    newrow['Decision Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            size+=1
    return newframe

In [135]:
t=splitRowEvents(event_assoc)

In [136]:
t.head()

,Decision People Involved,Decision Role
0,,Owner\n\nPurchaser
1,Amelia K. Kalogerakis,Purchaser
2,Eva K. Cleraderas,Purchaser
3,Steve Kalogerakis,Purchaser
4,,Purchaser


In [137]:
t['Decision People Involved'].replace('', np.nan, inplace=True)

In [138]:
t.head(20)

,Decision People Involved,Decision Role
0,NaN,Owner\n\nPurchaser
1,Amelia K. Kalogerakis,Purchaser
2,Eva K. Cleraderas,Purchaser
3,Steve Kalogerakis,Purchaser
4,NaN,Purchaser
5,HACA,Owner
6,Charles R. Stinnette Co.,Owner
7,NaN,Owner
8,HACA,Purchaser
9,William I. Lee,Owner


In [139]:
t.dropna(inplace=True)

In [140]:
t.columns = ["people","role"]

In [92]:
# t.people=t.people.apply(lambda x: x.lower())

In [141]:
for index, row in t.iterrows():
    name = name_cleaner(str(row["people"]))
    t.loc[t.index == index,"people"] = name

In [142]:
t.head(20)
# NOTE: people 1-5 have incorrect roles 

,people,role
1,Amelia K Kalogerakis,Purchaser
2,Eva K Cleraderas,Purchaser
3,Steve Kalogerakis,Purchaser
5,HACA,Owner
6,Charles R Stinnette Co,Owner
8,HACA,Purchaser
9,William I Lee,Owner
10,Zyphin J Lee,Owner
11,H Kenneth Lee,Owner
12,Frances W Lee,Owner


In [143]:
t = t.merge(newframe,how='left', left_on='people',right_on='name')


In [144]:
t.dropna(inplace=True)

In [145]:
t.head()

,people,role,name,person_id
0,Amelia K Kalogerakis,Purchaser,Amelia K Kalogerakis,928.0
1,Amelia K Kalogerakis,Purchaser,Amelia K Kalogerakis,929.0
2,Amelia K Kalogerakis,Purchaser,Amelia K Kalogerakis,930.0
3,Eva K Cleraderas,Purchaser,Eva K Cleraderas,3863.0
4,Eva K Cleraderas,Purchaser,Eva K Cleraderas,3864.0


In [ ]:
newframe.to_csv('output/people_count.csv')

# Event Type

In [98]:
# extracting types of events 
types=['Previous Transfer of Deed','Final Transfer of Deed','Appraisal','Offer Made', 'Decision', 'Award','Tenant','End of Case']

In [99]:
event_types=pd.DataFrame({'Type':types})

In [100]:
event_types

,Type
0,Previous Transfer of Deed
1,Final Transfer of Deed
2,Appraisal
3,Offer Made
4,Decision
5,Award
6,Tenant
7,End of Case


In [101]:
event_types.insert(0, 'ID', range(1,1+len(event_types)))

In [102]:
event_types.to_csv('output/event_types.csv')

# Previous Transfer 

In [103]:
previous_events=copy[['Block #', 'Parcel #','Previous Transfer of Deed Event Date','Previous Transfer of Deed People Involved','Previous Transfer of Deed Role']]

In [104]:
aa = str(previous_events.iloc[[44]]["Previous Transfer of Deed Event Date"])

In [105]:
def splitRowPreviousEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        
        parcel=row['Parcel #']
        block=row['Block #']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Previous Transfer of Deed People Involved']):
            
            # when the row is separated by ---        
            if '---' in str(row['Previous Transfer of Deed People Involved']):

                people = str(row['Previous Transfer of Deed People Involved'])
                roles = str(row['Previous Transfer of Deed Role'])
                agree_dates = str(row['Previous Transfer of deed Event Date'])
                
                people = re.sub(r'[\-]+', 'separator', people) 
                roles = re.sub(r'[\-]+', 'separator', roles) 
                agree_dates = re.sub(r'[\-]+', 'separator', agree_dates) 
                
                people = people.split("separator")
                roles = roles.split("separator")
                agree_dates = agree_dates.split("separator")
                
                prev_role = ""
                prev_date = ""
                
                for d in range(len(people)):
                    people2 = people[d].split('\n')
                    
                    # substitue characters by \n to then slit into new lines using \n
                    try:       
                        roles2 = re.sub(r'[\n]+', '\n', roles[d])                        
                        roles2 = roles2.strip().split('\n')
                        prev_role = roles2
                    except:
                        roles2 = prev_role
                        
                    try:                        
                        agree_dates2 = re.sub(r'[\n]+', '\n', agree_dates[d])                        
                        agree_dates2 = agree_dates2.strip().split('\n')
                        prev_date = agree_dates2
                    except:
                        agree_dates2 = prev_date                        
                    
                    
                    
                    role_counter = 0
                    date_counter = 0
                    
                    for k in range(len(people2)):
                        if people2[k] == '':
                            role_counter += 1
                            date_counter += 1

#                         name_cleaner: function which removes unwanted characters from the names 
                        newrow2 = row                
                        newrow2['Previous Transfer of Deed People Involved'] = name_cleaner(people2[k])
                            
                        try:
                            newrow2['Previous Transfer of Deed Role']= roles2[role_counter]
                        except:
                            role_counter -= 1
                            newrow2['Previous Transfer of Deed Role']= roles2[role_counter]

                        try:
                            newrow2['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])
                        except:
                            date_counter -= 1
                            newrow2['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])

                           
                        newrow2['Parcel #']=parcel
                        newrow2['Block #']=block                        

                        newframe.loc[size]=newrow2
                        size+=1
                
            # when the row is NOT separated by ---            
            else:
                roles = re.sub(r'[\n]+', '\n', str(row['Previous Transfer of Deed Role'])) 
                agree_dates = re.sub(r'[\n]+', '\n', str(row['Previous Transfer of Deed Event Date'])) 
                
                people = row['Previous Transfer of Deed People Involved'].split('\n')
                roles = roles.strip().split('\n')
                agree_dates = agree_dates.strip().split('\n')
                role_counter = 0
                date_counter = 0

                for d in range(len(people)):
                    if people[d] == '':
                        role_counter += 1
                        date_counter += 1

                    newrow = row                
                    newrow['Previous Transfer of Deed People Involved'] = name_cleaner(people[d])
                    try:
                        newrow['Previous Transfer of Deed Role']= roles[role_counter]
                    except:
                        role_counter -= 1
                        newrow['Previous Transfer of Deed Role']= roles[role_counter]
                        
                    try:
                        newrow['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                    except:
                        date_counter -= 1
                        newrow['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                        
                    newrow['Parcel #']=parcel
                    newrow['Block #']=block                        

                    newframe.loc[size]=newrow
                    size+=1
        else:
            if str(row["Previous Transfer of Deed People Involved"]) == "-" or str(row["Previous Transfer of Deed People Involved"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Previous Transfer of Deed People Involved"]))
                newframe.loc[newframe.index == size,"Previous Transfer of Deed People Involved"] = name
                date = date_cleaner(str(row["Previous Transfer of Deed Event Date"]))
                newframe.loc[newframe.index == size,"Previous Transfer of Deed Event Date"] = date
                
                
            size+=1
    return newframe


In [106]:
prev_events=splitRowPreviousEvents(previous_events)

In [107]:
prev_events['Previous Transfer of Deed People Involved'].replace('', np.nan, inplace=True)


In [108]:
prev_events=prev_events.dropna() 

In [109]:
prev_events

,Block #,Parcel #,Previous Transfer of Deed Event Date,Previous Transfer of Deed People Involved,Previous Transfer of Deed Role
0,1,2,-,Bessie S Kalogerakis,Owner
2,1,2,-,Amelia K Kalogerakis,Heir
3,1,2,-,Eva K Cleraderas,Heir
4,1,2,-,Steve Kalogerakis,Heir
5,1,3,8/1/1967,W S Harrison,Owner
...,...,...,...,...,...
3949,33,7,6/9/1966,Marion B Haynes Jr,Owner
3951,33,7,6/9/1966,Wachovia Bank Trust Co,Trustee
3952,33,10,8/16/1943,Carolyn Thrash Dorsett,Owner
3954,33,10,8/16/1943,Lucile Renwick,Purchaser


## Event Type 1: Previous Transper of Deed

In [110]:
# Assigning Event ID to Previous Transfer of Deed (after cleaning up that data)

In [111]:
prev_events.insert(0, 'Event_Type', 1)

In [112]:
prev_events.columns = ["event_types", "block_num", "parcel_num", "event_date", "person", "person_role"]

In [113]:
prev_events.head()

,event_types,block_num,parcel_num,event_date,person,person_role
0,1,1,2,-,Bessie S Kalogerakis,Owner
2,1,1,2,-,Amelia K Kalogerakis,Heir
3,1,1,2,-,Eva K Cleraderas,Heir
4,1,1,2,-,Steve Kalogerakis,Heir
5,1,1,3,8/1/1967,W S Harrison,Owner


In [151]:
prev_events.to_csv("output/prev_events.csv")

## Event Type 2: Final Transfer of Deed

In [152]:
final_events=copy[['Block #', 'Parcel #','Final Transfer of Deed Event Date','Final Transfer of Deed Amount Money','Final Transfer of Deed People Involved',
                   'Final Transfer of Deed Role']]

In [153]:
final_events.head()

,Block #,Parcel #,Final Transfer of Deed Event Date,Final Transfer of Deed Amount Money,Final Transfer of Deed People Involved,Final Transfer of Deed Role
0,1,2,6/6/1969,8500,Eva K. Cledaras\nGeorge M. Cledaras\nSteve Kal...,Owner\n\nPurchaser
1,1,3,6/17/1971,15000,Charles R. Stinnette Co.\n\nHACA,Owner\n\nPurchaser
2,1,4,7/13/1972,46000,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Owner\n\nPurchaser
3,1,5,5/29/1974,8000,Thelma Smathers Johnson\n\nHACA,Owner\n\nPurchaser
4,1,6,10/9/1974,40000,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,Owner\n\nPurchaser


In [154]:
def splitRowFinalEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Final Transfer of Deed Amount Money']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Final Transfer of Deed People Involved']):
            
            # when the row is separated by ---        
            if '---' in str(row['Final Transfer of Deed People Involved']):

                people = str(row['Final Transfer of Deed People Involved'])
                roles = str(row['Final Transfer of Deed Role'])
                agree_dates = str(row['Final Transfer of Deed Event Date'])
                
                people = re.sub(r'[\-]+', 'separator', people) 
                roles = re.sub(r'[\-]+', 'separator', roles) 
                agree_dates = re.sub(r'[\-]+', 'separator', agree_dates) 
                
                people = people.split("separator")
                roles = roles.split("separator")
                agree_dates = agree_dates.split("separator")
                
                prev_role = ""
                prev_date = ""
                
                for d in range(len(people)):
                    people2 = people[d].split('\n')
                    
                    try:                        
                        roles2 = re.sub(r'[\n]+', '\n', roles[d])                        
                        roles2 = roles2.strip().split('\n')
                        prev_role = roles2
                    except:
                        roles2 = prev_role
                        
                    try:                        
                        agree_dates2 = re.sub(r'[\n]+', '\n', agree_dates[d])                        
                        agree_dates2 = agree_dates2.strip().split('\n')
                        prev_date = agree_dates2
                    except:
                        agree_dates2 = prev_date                        
                    
                    
                    
                    role_counter = 0
                    date_counter = 0
                    
                    for k in range(len(people2)):
                        if people2[k] == '':
                            role_counter += 1
                            date_counter += 1

                        newrow2 = row                
                        newrow2['Final Transfer of Deed People Involved'] = name_cleaner(people2[k])
                            
                        try:
                            newrow2['Final Transfer of Deed Role']= roles2[role_counter]
                        except:
                            role_counter -= 1
                            newrow2['Final Transfer of Deed Role']= roles2[role_counter]

                        try:
                            newrow2['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])
                        except:
                            date_counter -= 1
                            newrow2['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])

                           
                        newrow2['Parcel #']=parcel
                        newrow2['Block #']=block   
                        newrow2['Final Transfer of Deed Amount Money'] = money

                        newframe.loc[size]=newrow2
                        size+=1
                
            # when the row is NOT separated by ---            
            else:
                roles = re.sub(r'[\n]+', '\n', str(row['Final Transfer of Deed Role'])) 
                agree_dates = re.sub(r'[\n]+', '\n', str(row['Final Transfer of Deed Event Date'])) 
                
                people = row['Final Transfer of Deed People Involved'].split('\n')
                roles = roles.strip().split('\n')
                agree_dates = agree_dates.strip().split('\n')
                role_counter = 0
                date_counter = 0

                for d in range(len(people)):
                    if people[d] == '':
                        role_counter += 1
                        date_counter += 1

                    newrow = row                
                    newrow['Final Transfer of Deed People Involved'] = name_cleaner(people[d])
                    try:
                        newrow['Final Transfer of Deed Role']= roles[role_counter]
                    except:
                        role_counter -= 1
                        newrow['Final Transfer of Deed Role']= roles[role_counter]
                        
                    try:
                        newrow['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                    except:
                        date_counter -= 1
                        newrow['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                        
                    newrow['Parcel #']=parcel
                    newrow['Block #']=block     
                    newrow['Final Transfer of Deed Amount Money'] = money

                    newframe.loc[size]=newrow
                    size+=1
        else:
            if str(row["Final Transfer of Deed People Involved"]) == "-" or str(row["Final Transfer of Deed People Involved"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Final Transfer of Deed People Involved"]))
                newframe.loc[newframe.index == size,"Final Transfer of Deed People Involved"] = name
                date = date_cleaner(str(row["Final Transfer of Deed Event Date"]))
                newframe.loc[newframe.index == size,"Final Transfer of Deed Event Date"] = date
                
                
            size+=1
    return newframe

In [155]:
final_events=splitRowFinalEvents(final_events)

In [156]:
final_events['Final Transfer of Deed People Involved'].replace('', np.nan, inplace=True)


In [157]:
final_events=final_events.dropna()

In [158]:
# Assigning Type_id 2
final_events.insert(0, 'Event_Type', 2)

In [159]:
final_events.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "person", "person_role"]

In [160]:
final_events.head()

,event_types,block_num,parcel_num,event_date,money,person,person_role
0,2,1,2,6/6/1969,8500,Eva K Cledaras,Owner
1,2,1,2,6/6/1969,8500,George M Cledaras,Owner
2,2,1,2,6/6/1969,8500,Steve Kalogerakis,Owner
3,2,1,2,6/6/1969,8500,Shirley Kalogerakis,Owner
4,2,1,2,6/6/1969,8500,Amelia K Candler,Owner


In [161]:
final_events.to_csv("output/final_events.csv")

## Event Type 3: Appraisal

In [162]:
appraisel=copy[['Block #', 'Parcel #','Appraiser Name','Appraiser Event Date','Appraiser Amount Money']]

In [163]:
appraisel.head()

,Block #,Parcel #,Appraiser Name,Appraiser Event Date,Appraiser Amount Money
0,1,2,Clyde G. Idol Jr.\nAlan Butterworth\nH.E. Johnson,09/15/1965\n07/31/1968\n11/09/1968,6250\n7000\n9300
1,1,3,Charles W. Bradshaw\nAlan Butterworth\nH.E. Jo...,09/15/1965\n07/12/1968\n11/02/1968,16400\n15000\n14000
2,1,4,Charles W. Bradshaw\nAlan Butterworth\nH.E. Jo...,09/15/1965\n07/15/1968\n11/02/1968,27000\n35000\n35500
3,1,5,Clyde G. Idol Jr.\nAlan Butterworth\nH.E. Johnson,09/15/1965\n07/16/1968\n10/22/1968,7250\n8000\n6600
4,1,6,Edward Sale\nRoy Moore\nH.E. Johnson,04/14/1974\n08/09/1974\n03/04/1974,43706\n40000\n38500


In [164]:
exceptionList=[]
def splitRowAppraise(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        parcel=row['Parcel #']
        block=row['Block #']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Appraiser Name']):
            data = row['Appraiser Name'].split('\n')
            dates=str(row['Appraiser Event Date']).split('\n')
            money=str(row['Appraiser Amount Money']).split('\n')
            placeholder=0
            try:
                for d in range(len(data)):
                    newrow = row
                    newrow['Appraiser Name'] = name_cleaner(data[d])
                    newrow['Appraiser Event Date']= dates[d]
                    if d < len(money):
                        newrow['Appraiser Amount Money']=  money[d]
                        if placeholder!=0:
                            placeholder += 1
                    else:
                        newrow['Appraiser Amount Money']=  money[placeholder]
                    newrow['Parcel #']= parcel
                    newrow['Block #']= block
                    newframe.loc[size]=newrow
                    size+=1
            except:
                exceptionList.append(row) 
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Appraiser Name"]))
            newframe.loc[newframe.index == size,"Appraiser Name"] = name
            size+=1
    return newframe
 

In [165]:
appraisel = splitRowAppraise(appraisel)

In [166]:
appraisel.insert(0, 'Event_Type', 3)

In [167]:
print(exceptionList)

[Block #                            4
Parcel #                          20
Appraiser Name             Roy Moore
Appraiser Event Date      01/29/1975
Appraiser Amount Money         13000
Name: 88, dtype: object, Block #                            4
Parcel #                          23
Appraiser Name                      
Appraiser Event Date      09/15/1965
Appraiser Amount Money          7850
Name: 91, dtype: object, Block #                            4
Parcel #                          24
Appraiser Name                      
Appraiser Event Date      09/15/1965
Appraiser Amount Money          9000
Name: 92, dtype: object, Block #                            5
Parcel #                           2
Appraiser Name                      
Appraiser Event Date      09/15/1965
Appraiser Amount Money          4400
Name: 95, dtype: object, Block #                            5
Parcel #                          16
Appraiser Name                      
Appraiser Event Date      09/15/1965
Appraiser A

In [168]:
appraisel.columns = ["event_types", "block_num", "parcel_num", "person", "event_date", "money"]

In [169]:
appraisel.head()

,event_types,block_num,parcel_num,person,event_date,money
0,3,1,2,Clyde G Idol Jr,09/15/1965,6250
1,3,1,2,Alan Butterworth,07/31/1968,7000
2,3,1,2,H E Johnson,11/09/1968,9300
3,3,1,3,Charles W Bradshaw,09/15/1965,16400
4,3,1,3,Alan Butterworth,07/12/1968,15000


In [170]:
appraisel.to_csv("output/appraisel_events.csv")

## Event Type 4: Offer Made

In [171]:
offerMade=copy[['Block #', 'Parcel #','Offer Made Amount Money','Offer Made People Involved','Offer Made Event Date','Offer Made Role']]

In [172]:
offerMade.head()

,Block #,Parcel #,Offer Made Amount Money,Offer Made People Involved,Offer Made Event Date,Offer Made Role
0,1,2,8500,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...,3/14/1969,Owner\n\nPurchaser
1,1,3,15000,Charles R. Stinnette Co.\n\nHACA,8/6/1969,Owner\n\nPurchaser
2,1,4,35500,Willian I. Lee\nZyphin J. Lee\nH. Kenneth Lee\...,6/17/1970,Owner\n\nPurchaser
3,1,5,8000,Thelma Smathers Johnson\nElizabeth Pauline Sma...,11/17/1969,Owner\n\nPurchaser
4,1,6,40000,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,8/16/1974,Owner\n\nPurchaser


In [173]:
def splitRowOfferMade(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date=row['Offer Made Event Date']
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Offer Made Amount Money']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Offer Made People Involved']):
            data = row['Offer Made People Involved'].split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            for d in range(len(data)):
                newrow = row
                newrow['Offer Made People Involved'] = name_cleaner(data[d])
                newrow['Offer Made Event Date']= date
                newrow['Parcel #']=parcel
                newrow['Block #']=block
                newrow['Offer Made Amount Money']=money
                if data[d]!='':
                    newrow['Offer Made Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Offer Made People Involved"]))
            newframe.loc[newframe.index == size,"Offer Made People Involved"] = name
            size+=1
    return newframe

In [174]:
offerMade=splitRowOfferMade(offerMade)
##there is an extra newline at the begining that messes up the first row

In [175]:
offerMade['Offer Made People Involved'].replace('', np.nan, inplace=True)


In [176]:
offerMade=offerMade.dropna()

In [177]:
offerMade.insert(0, 'Event_Type', 4)

In [178]:
offerMade.columns = ["event_types", "block_num", "parcel_num", "money", "person", "event_date", "person_role"]

In [179]:
offerMade.head()

,event_types,block_num,parcel_num,money,person,event_date,person_role
1,4,1,2,8500,Amelia K Kalogerakis,3/14/1969,Purchaser
2,4,1,2,8500,Eva K Cleraderas,3/14/1969,Purchaser
3,4,1,2,8500,Steve Kalogerakis,3/14/1969,Purchaser
5,4,1,2,8500,HACA,3/14/1969,Owner
6,4,1,3,15000,Charles R Stinnette Co,8/6/1969,Owner


In [180]:
offerMade.to_csv("output/offer_events.csv")

## Event Type 5: Decision Made

In [181]:
decision=copy[['Block #', 'Parcel #','Decision Event Date','Decision Amount Money','Decision Response','Decision People Involved','Decision Role']]

In [182]:
decision.head()

,Block #,Parcel #,Decision Event Date,Decision Amount Money,Decision Response,Decision People Involved,Decision Role
0,1,2,3/14/1969,8500,Accepted,\nAmelia K. Kalogerakis\nEva K. Cleraderas\nSt...,Owner\n\nPurchaser
1,1,3,8/6/1969,21600,Rejected,Charles R. Stinnette Co.\n\nHACA,Owner\n\nPurchaser
2,1,4,6/17/1970,35500,Rejected,William I. Lee\nZyphin J. Lee\nH. Kenneth Lee\...,Owner\n\nPurchaser
3,1,5,5/7/1974,8000,Accepted,Thelma Smathers Johnson\nElizabeth Pauline Sma...,Owner\n\nPurchaser
4,1,6,10/9/1974,40000,Accepted,Harry's Cadillac-Pontiac Company Inc.\n\nHACA\n,Owner\n\nPurchaser


In [183]:
def splitRowDecision(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date=row['Decision Event Date']
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Decision Amount Money']
        response=row['Decision Response']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Decision People Involved']):
            data = row['Decision People Involved'].split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            for d in range(len(data)):
                newrow = row
                newrow['Decision People Involved'] = name_cleaner(data[d])
                newrow['Decision Event Date']= date
                newrow['Parcel #']=parcel
                newrow['Block #']=block
                newrow['Decision Amount Money']=money
                newrow['Decision Response']=response
                if data[d]!='':
                    newrow['Decision Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Decision People Involved"]))
            newframe.loc[newframe.index == size,"Decision People Involved"] = name
            size+=1
    return newframe

In [184]:
decision=splitRowDecision(decision)

decision['Decision People Involved'].replace('', np.nan, inplace=True)

decision=decision.dropna()

decision.insert(0,'Event_Type',5)

decision.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "response", "person", "person_role"]

decision.head()

,event_types,block_num,parcel_num,event_date,money,response,person,person_role
1,5,1,2,3/14/1969,8500,Accepted,Amelia K Kalogerakis,Purchaser
2,5,1,2,3/14/1969,8500,Accepted,Eva K Cleraderas,Purchaser
3,5,1,2,3/14/1969,8500,Accepted,Steve Kalogerakis,Purchaser
5,5,1,2,3/14/1969,8500,Accepted,HACA,Owner
6,5,1,3,8/6/1969,21600,Rejected,Charles R Stinnette Co,Owner


In [185]:
decision.to_csv("output/decision_events.csv")

## Event Type 6: Award

In [186]:
award=copy[['Block #', 'Parcel #','Award Date','Award Amount Money']]

In [187]:
award.head()

,Block #,Parcel #,Award Date,Award Amount Money
0,1,2,-,-
1,1,3,7/15/1971,21600
2,1,4,8/17/1972,46000
3,1,5,-,-
4,1,6,-,-


In [188]:
award.insert(0,'Event_Type',6)

In [189]:
award.columns = ["event_types", "block_num", "parcel_num", "event_date", "money"]

In [190]:
award.head()

,event_types,block_num,parcel_num,event_date,money
0,6,1,2,-,-
1,6,1,3,7/15/1971,21600
2,6,1,4,8/17/1972,46000
3,6,1,5,-,-
4,6,1,6,-,-


In [191]:
award.to_csv("output/award_events.csv")

## Event Type 7: Tenant

In [192]:
tenant=copy[['Block #', 'Parcel #','Tenant Vacated Date','Tenant Agreement Date','Tenant Cost','Tenant Name','Previous owner\nor\nTenant']]

In [193]:
tenant.head()

,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous owner\nor\nTenant
0,1,2,-,-,-,-,-
1,1,3,-,-,-,-,-
2,1,4,-,-,-,-,-
3,1,5,-,-,-,-,-
4,1,6,-,-,-,-,-


### Needs to write a robust code to tease out names and their roles here

In [194]:
def splitRowTenant(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date= row['Tenant Vacated Date'] if row['Tenant Vacated Date'] != "-" else ""
        parcel=row['Parcel #']
        block=row['Block #']
#         money=row['Tenant Cost']
#         role=row['Previous owner or Tenant']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Tenant Name']):
            tenants = row['Tenant Name'].split('\n')
            rents = row['Tenant Cost'].split('\n')
            roles = row['Previous owner\nor\nTenant'].split('\n')
            agree_dates = row['Tenant Agreement Date'].split('\n')
            prev_role = ""
            prev_date = ""

            for d in range(len(tenants)):
                if tenants[d] == '':
                    continue
                
                newrow = row                
                newrow['Tenant Name'] = name_cleaner(tenants[d])
                newrow['Tenant Cost'] = re.sub(r'[\$]', '', rents[d])
                
                try:
                    newrow['Previous owner\nor\nTenant']= roles[d]
                    prev_role = roles[d] 
                except:
                    newrow['Previous owner\nor\nTenant'] = prev_role
                
                try:
                    newrow['Tenant Agreement Date']= agree_dates[d]
                    prev_date = agree_date[d]
                except:
                    newrow['Tenant Agreement Date']= prev_date
                    
                newrow['Tenant Vacated Date']= date                
                newrow['Parcel #']=parcel
                newrow['Block #']=block                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            if str(row["Tenant Name"]) == "-" or str(row["Tenant Name"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Tenant Name"]))
                newframe.loc[newframe.index == size,"Tenant Name"] = name
                
                try:
                    rent = re.sub(r'[\$\-]', '', str(row['Tenant Cost']))
                    newframe.loc[newframe.index == size,"Tenant Cost"] = rent
                except Exception as e: 
                    print(e)
                    print (row)
                    break
                
            size+=1
    return newframe

In [195]:
tenant.insert(0,'Event_Type',7)

In [196]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous owner\nor\nTenant
0,7,1,2,-,-,-,-,-
1,7,1,3,-,-,-,-,-
2,7,1,4,-,-,-,-,-
3,7,1,5,-,-,-,-,-
4,7,1,6,-,-,-,-,-


In [197]:
tenant=splitRowTenant(tenant)

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying 

IndexError: list index out of range

In [ ]:
tenant['Tenant Name'].replace('', np.nan, inplace=True)
tenant=tenant.dropna()

In [ ]:
tenant.columns = ["event_types", "block_num", "parcel_num", "extra_information", "event_date", "money", "person", "person_role"]


In [ ]:
tenant.event_types = tenant["event_types"].astype(int)

In [ ]:
tenant.head()

In [ ]:
tenant.to_csv("output/tenant_events.csv")

## Event Type 8: End of Case

In [199]:
endCase=copy[['Block #', 'Parcel #','End of Case Finalized Date']]

In [200]:
endCase.head()

,Block #,Parcel #,End of Case Finalized Date
0,1,2,6/6/1969
1,1,3,6/17/1971
2,1,4,7/13/1972
3,1,5,5/29/1974
4,1,6,10/9/1974


In [201]:
endCase.insert(0,'Event_Type',8)

In [202]:
endCase.columns = ["event_types", "block_num", "parcel_num", "event_date"]

In [203]:
endCase.head()

,event_types,block_num,parcel_num,event_date
0,8,1,2,6/6/1969
1,8,1,3,6/17/1971
2,8,1,4,7/13/1972
3,8,1,5,5/29/1974
4,8,1,6,10/9/1974


In [204]:
endCase.to_csv("output/end_events.csv")

## Event Type 9: Extra Information

In [260]:
# collecting all the extra information into one table
notes = copy[['Address Notes','Previous Transfer of Deed Notes','Final Transfer of Deed Notes','Appraiser Notes','Offer Made Notes','Decision Notes','Award Notes','Tenant Notes']]

In [261]:
notes.head()

,Address Notes,Previous Transfer of Deed Notes,Final Transfer of Deed Notes,Appraiser Notes,Offer Made Notes,Decision Notes,Award Notes,Tenant Notes
0,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,-,-,-
1,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,"Concurred price was $15,000.",There was an interest amount of $536.39. The d...,-
2,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,"Concurred price was $35,500.","There was an interest amount of $1,778.68. The...",-
3,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,-,-,-
4,-,-,-,-,-,This case was assigned to an attorney on 09/24...,The award date and amount are unclear because ...,-


In [262]:
# this methods combines all the columns of extra info into one column separated by ','
def combineNotes(table):

    newframe= pd.DataFrame(columns=table.columns.values)
    string_combined=table.stack().groupby(level=0).apply(','.join)
    newframe=string_combined
    return newframe

In [263]:
combining= combineNotes(notes)

In [270]:
combining.head()


0    -,-,-,On the appraisal document, the name "Cly...
1    -,-,-,On the appraisal document, the name "Cly...
2    -,-,-,On the appraisal document, the name "Cly...
3    -,-,-,On the appraisal document, the name "Cly...
4    -,-,-,-,-,This case was assigned to an attorne...
dtype: object

In [268]:
combining.to_frame(combining)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [258]:
# def note_cleaner(table):
#     newframe= pd.DataFrame(columns= table.columns.value)
#     size= 0 
#     for i in range(len(table)):
#         row= table.iloc[i]
#         parcel=row['Parcel #']
#         block=row['Block #']
        
#         newfram.loc[size]=row
#         notes = name_cleaner(str(row['Extra_Information']))
#         newframe.loc[newframe.index == size, 'Extra_Information'] =notes
        
#         size +=1
#     return newframe

In [284]:
def note_cleaner(table):
    clean_notes=[]
    
    for index in table.iteritems():
        notes = str(row(table[1])
        
        if ',-' in notes
            k = re.sub(r'[0-9][.]', '', notes)
            clean_notes.append(k)
    return clean_notes

SyntaxError: invalid syntax (<ipython-input-284-278401a4f837>, line 8)

In [280]:
clean_combined= note_cleaner(combining) 

In [282]:
print(clean_combined)

[]


In [ ]:
extra_info= copy[['Block #', 'Parcel #']]

In [247]:
extra_info.insert(2, 'Extra_Information',combining)

In [ ]:
extra_info.head()

In [ ]:
extra_info.insert(0, 'Event_Type',9)

In [177]:
checking.columns = ["event_types", "block_num", "parcel_num", "notes"]

In [174]:
# extra_info.columns = ["event_types", "block_num", "parcel_num", "address_notes", "previous_notes","final_notes","appraiser_notes","offer_notes","decision_notes","award_notes","tenant_notes"]

In [175]:
extra_info.head()

,event_types,block_num,parcel_num,address_notes,previous_notes,final_notes,appraiser_notes,offer_notes,decision_notes,award_notes,tenant_notes
0,9,1,2,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,-,-,-
1,9,1,3,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,"Concurred price was $15,000.",There was an interest amount of $536.39. The d...,-
2,9,1,4,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,"Concurred price was $35,500.","There was an interest amount of $1,778.68. The...",-
3,9,1,5,-,-,-,"On the appraisal document, the name ""Clyde G. ...",-,-,-,-
4,9,1,6,-,-,-,-,-,This case was assigned to an attorney on 09/24...,The award date and amount are unclear because ...,-


In [156]:
extra_info.to_csv("output/extra_info.csv")

## Concatenating all Events

In [252]:
events_table = pd.concat([prev_events,final_events,appraisel,offerMade,decision,award,tenant, endCase,])

In [253]:
events_table.insert(0, 'event_id', range(1,1+len(events_table)))

In [254]:
events_table.head()

,event_id,event_types,block_num,parcel_num,event_date,person,person_role,money,response,Event_Type,...,Tenant Name,Previous owner\nor\nTenant,address_notes,previous_notes,final_notes,appraiser_notes,offer_notes,decision_notes,award_notes,tenant_notes
0,1,1.0,1,2,-,Bessie S Kalogerakis,Owner,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.0,1,2,-,Amelia K Kalogerakis,Heir,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.0,1,2,-,Eva K Cleraderas,Heir,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.0,1,2,-,Steve Kalogerakis,Heir,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1.0,1,3,8/1/1967,W S Harrison,Owner,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
events_table.to_csv('output/events.csv')

# People Clean up (Bez)

1. Condence repeated names by reading old ID and assigning it to a new ID

In [60]:
condenced_people = pd.read_csv("output/people_clean_final2.csv")

In [61]:
condenced_people.head(15)

,Original name,person_id,Same Person ID
0,#Name,1,NaN
1,*Not Legable,2,NaN
2,A Blomberg,3,NaN
3,A C Brandl,4,NaN
4,A C Brandl Jr,"5, 6, 7",NaN
5,A C Williams,8,NaN
6,A Carrell Jelliff,"9, 10",NaN
7,A G Caever Sr,"11, 12, 13",NaN
8,A G Carver,"14, 15, 16, 17, 18, 19",NaN
9,A G Carver Jr,"20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31...",NaN


In [62]:
condenced_people= condenced_people[["Original name", "person_id"]]

In [ ]:
## person_id_cleaner is a method which removes all empty spaces from the ids

In [64]:
def person_id_cleaner(table): 
    clean_id=[]
    
    for index, row in table.iterrows():
        id= str(row['person_id'])
        
        k = re.sub(' ', '', id)
        k = k.strip()
        if k== '':
            continue
        clean_id.append(k)
    return clean_id

In [65]:
id_clean= person_id_cleaner(condenced_people)

In [67]:
new_people_frame= pd.DataFrame(columns=['id', 'name'])

In [68]:
new_people_frame['id']= pd.Series(id_clean)

In [69]:
new_people_frame['name']= condenced_people['Original name']

In [70]:
new_people_frame.head(15)

,id,name
0,1,#Name
1,2,*Not Legable
2,3,A Blomberg
3,4,A C Brandl
4,"5,6,7",A C Brandl Jr
5,8,A C Williams
6,"9,10",A Carrell Jelliff
7,"11,12,13",A G Caever Sr
8,"14,15,16,17,18,19",A G Carver
9,"20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35",A G Carver Jr


## People Cleaned Up (Dr Lee)

In [161]:
new_people = pd.read_csv("output/people_count_cleaned.csv")

In [162]:
new_people.head()

,id,name,count,real-name,Unnamed: 4
0,1,?,2,?,NaN
1,2,A C Brandl,2,A C Brandl,NaN
2,3,A C Brandl Jr,3,A C Brandl Jr,NaN
3,4,A Carrell Jelliff,2,A Carrell Jelliff,NaN
4,5,A G Carver,6,A G Carver,NaN


In [163]:
new_people = new_people[["id", "name", "real-name"]]

In [164]:
real_name = new_people.groupby("name").agg("count")

In [165]:
real_name["name"] = real_name.index

In [166]:
real_name.id = range(1,1+len(real_name))

In [167]:
real_name.index = range(1,1+len(real_name))

In [168]:
real_name.head()

,id,real-name,name
1,1,1,?
2,2,1,A C Brandl
3,3,1,A C Brandl Jr
4,4,1,A Carrell Jelliff
5,5,1,A G Carver


In [169]:
real_name.to_csv("output/real_people.csv")

In [170]:
new_people = new_people.merge(real_name, how='left', left_on='real-name', right_on='name')

In [171]:
new_people = new_people[["id_x", "name_x", "id_y", "name_y"]]

In [172]:
new_people.columns = ["id", "name", "realname_id", "realname"]

# Join other tables based on the Event table

In [2]:
##new_people.to_csv("output/people.csv", index=False)

NameError: name 'new_people' is not defined

In [174]:
##new_people = pd.read_csv("output/People.csv")

In [175]:
##new_people.head()

,id,name,realname_id,realname
0,1,?,1.0,?
1,2,A C Brandl,2.0,A C Brandl
2,3,A C Brandl Jr,3.0,A C Brandl Jr
3,4,A Carrell Jelliff,4.0,A Carrell Jelliff
4,5,A G Carver,5.0,A G Carver


In [146]:
events_table = pd.read_csv("output/final_events.csv")

In [147]:
events_table.head()

,Unnamed: 0,event_types,block_num,parcel_num,event_date,money,person,person_role
0,0,2,1,2,6/6/1969,8500,Eva K Cledaras,Owner
1,1,2,1,2,6/6/1969,8500,George M Cledaras,Owner
2,2,2,1,2,6/6/1969,8500,Steve Kalogerakis,Owner
3,3,2,1,2,6/6/1969,8500,Shirley Kalogerakis,Owner
4,4,2,1,2,6/6/1969,8500,Amelia K Candler,Owner


In [160]:
newframe.head()

,name,person_id
0,#NAME?,1
1,*Not Legable,2
2,A Blomberg,3
3,A C Brandl,4
4,A C Brandl Jr,5


In [157]:
events_table_test = events_table.merge(newframe, how='left', left_on='person', right_on='name')

In [158]:
events_table_test.head()

,Unnamed: 0,event_types,block_num,parcel_num,event_date,money,person,person_role,id,name_x,name_y,person_id
0,0,2,1,2,6/6/1969,8500,Eva K Cledaras,Owner,3862,Eva K Cledaras,Eva K Cledaras,3862.0
1,1,2,1,2,6/6/1969,8500,George M Cledaras,Owner,4652,George M Cledaras,George M Cledaras,4652.0
2,2,2,1,2,6/6/1969,8500,Steve Kalogerakis,Owner,"12647,12648,12649,12650",Steve Kalogerakis,Steve Kalogerakis,12647.0
3,2,2,1,2,6/6/1969,8500,Steve Kalogerakis,Owner,"12647,12648,12649,12650",Steve Kalogerakis,Steve Kalogerakis,12648.0
4,2,2,1,2,6/6/1969,8500,Steve Kalogerakis,Owner,"12647,12648,12649,12650",Steve Kalogerakis,Steve Kalogerakis,12649.0


In [162]:
##events_table_test.to_csv('output/events_test.csv')

In [148]:
##events_table = events_table.merge(new_people_frame, how='left', left_on='person', right_on='name')

In [155]:
##events_table.head()

In [154]:
##events_table.to_csv('output/event_association.csv', index=False)

In [153]:
event_assc = events_table[["id", "realname_id", "person_role"]]

KeyError: "['realname_id'] not in index"

In [182]:
event_assc = event_assc.sort_values(by='realname_id', ascending=True)

In [183]:
event_assc.head()

,id,realname_id,person_role
1047,3.0,3.0,Owner
1218,5.0,5.0,Owner
387,14.0,14.0,Owner
386,14.0,14.0,Owner
384,14.0,14.0,Owner


In [184]:
event_assc.insert(0,'id',range(1,1+len(event_assc)))

ValueError: cannot insert id, already exists

In [142]:
event_assc.dropna(inplace=True)

In [144]:
event_assc["realname_id"] = event_assc["realname_id"].astype(int)

In [145]:
event_assc.to_csv('output/event_association.csv', index=False)

In [ ]:
#After this, I manually removed rows that don't have realname_ids

## Association Table Clean-up

In [1301]:
assoc = pd.read_csv("output/event_association.csv")

In [1302]:
assoc["realname_id"] = assoc.realname_id.astype(int)

In [1303]:
assoc.head()

,Unnamed: 0,id,event_id,realname_id,person_role
0,4878,1,4876,1,Owner
1,6164,2,6161,1,Owner
2,155,3,156,2,Owner
3,1328,4,1328,2,Purchaser
4,6939,5,6936,3,Owner


In [1304]:
assoc = assoc[["id", "event_id", "realname_id", "person_role"]]

In [1308]:
assoc.to_csv('output/event_association.csv', index=False)

## Linking Parcel IDs

In [107]:
events_table = pd.read_csv("output/Events.csv")

In [108]:
events_table.head()

,Unnamed: 0,event_id,block_num,event_date,event_types,extra_information,money,parcel_num,person,person_role,response
0,0,1,1,NaN,1,NaN,NaN,2,Bessie S Kalogerakis,Owner,NaN
1,2,2,1,NaN,1,NaN,NaN,2,Amelia K Kalogerakis,Heir,NaN
2,3,3,1,NaN,1,NaN,NaN,2,Eva K Cleraderas,Heir,NaN
3,4,4,1,NaN,1,NaN,NaN,2,Steve Kalogerakis,Heir,NaN
4,5,5,1,8/1/67,1,NaN,NaN,3,W S Harrison,Owner,NaN


In [109]:
parcel = pd.read_csv("output/Parcels.csv")

In [110]:
parcel.head()

,parcel_id,block_num,parcel_num,ward_num,land_use
0,1,1,2,4,Commercial
1,2,1,3,4,Commercial
2,3,1,4,4,Commercial
3,4,1,5,4,Residential
4,5,1,6,4,Commercial


In [111]:
events_table = events_table.merge(parcel, how='left', left_on=['block_num','parcel_num'], right_on=['block_num','parcel_num'])

In [112]:
events_table.head()

,Unnamed: 0,event_id,block_num,event_date,event_types,extra_information,money,parcel_num,person,person_role,response,parcel_id,ward_num,land_use
0,0,1,1,NaN,1,NaN,NaN,2,Bessie S Kalogerakis,Owner,NaN,1,4,Commercial
1,2,2,1,NaN,1,NaN,NaN,2,Amelia K Kalogerakis,Heir,NaN,1,4,Commercial
2,3,3,1,NaN,1,NaN,NaN,2,Eva K Cleraderas,Heir,NaN,1,4,Commercial
3,4,4,1,NaN,1,NaN,NaN,2,Steve Kalogerakis,Heir,NaN,1,4,Commercial
4,5,5,1,8/1/67,1,NaN,NaN,3,W S Harrison,Owner,NaN,2,4,Commercial


In [113]:
events = events_table[["event_id", "response", "extra_information", "parcel_id","event_date","event_types","money", "person", "person_role"]]

In [124]:
events["eid"] = range(1,1+len(events))

In [127]:
events.to_csv("output/final_events.csv", index=False)

In [126]:
events.tail()


,event_id,response,extra_information,parcel_id,event_date,event_types,money,person,person_role,eid
9961,8787,NaN,NaN,541,NaN,8,NaN,NaN,NaN,9962
9962,8787,NaN,NaN,542,NaN,8,NaN,NaN,NaN,9963
9963,8788,NaN,NaN,540,NaN,8,NaN,NaN,NaN,9964
9964,8788,NaN,NaN,541,NaN,8,NaN,NaN,NaN,9965
9965,8788,NaN,NaN,542,NaN,8,NaN,NaN,NaN,9966


In [146]:
final_events = events_table[["eid", "response", "extra_information", "parcel_id","event_date","event_types","money"]]

In [147]:
final_events.to_csv("output/db_events.csv", index=False)